In [1]:
import sys,os
sys.path.insert(0, os.path.abspath(".."))

In [2]:
import pandas as pd
from matplotlib import pyplot as plt
import os

In [3]:
import pycaret
import sklearn
import pandas as pd
print(f'pandas:{pd.__version__}')
print(f'sklearn:{sklearn.__version__}')
print(f'pycaret:{pycaret.__version__}')

pandas:2.1.4
sklearn:1.4.2
pycaret:3.3.2


In [4]:
target='formation_energy_per_site'
cfid_path = f'../results_2/2dmd/CatBoostRegressor/{target}/cfid/all_density/'
X_train = pd.read_csv(os.path.join(cfid_path, 'X_train.csv'),index_col=0)
X_test = pd.read_csv(os.path.join(cfid_path, 'X_test.csv'),index_col=0)
y_train = pd.read_csv(os.path.join(cfid_path, 'y_train.csv'),index_col=0)
y_test = pd.read_csv(os.path.join(cfid_path, 'y_test.csv'),index_col=0)

In [5]:

sample_weight = pd.read_csv(os.path.join(cfid_path, 'sample_weight.csv'),index_col=0)

In [6]:
test_data = pd.merge(X_test, y_test, left_index=True, right_index=True)
train_data = pd.merge(X_train, y_train, left_index=True, right_index=True)

In [7]:
from pycaret.regression import *
s = RegressionExperiment()
s.setup(data=train_data, target = target,
         session_id = 42,
        train_size=0.8,fold=5, #feature_selection=True,
        n_jobs=-1)

,Description,Value
0,Session id,42
1,Target,formation_energy_per_site
2,Target type,Regression
3,Original data shape,"(11892, 1558)"
4,Transformed data shape,"(11892, 1558)"
5,Transformed train set shape,"(9513, 1558)"
6,Transformed test set shape,"(2379, 1558)"
7,Numeric features,1557
8,Preprocess,True
9,Imputation type,simple


In [8]:
best =s.compare_models(n_select=10, sort = "MAE",include=[
    'et', 'rf', 'catboost', 'lightgbm',
    'dt', 'gbr', 'knn', 'ada', 'lr'
])

,Model,MAE,MSE,RMSE,R2,RMSLE,MAPE,TT (Sec)
et,Extra Trees Regressor,0.0478,0.0188,0.1364,0.9894,0.0411,0.0342,15.4600
rf,Random Forest Regressor,0.0529,0.0203,0.1422,0.9884,0.0457,0.0796,25.1560
catboost,CatBoost Regressor,0.0600,0.0187,0.1362,0.9894,0.0428,0.0645,37.0800
lightgbm,Light Gradient Boosting Machine,0.0619,0.0220,0.1480,0.9875,0.0464,0.0988,3.4260
dt,Decision Tree Regressor,0.0664,0.0377,0.1941,0.9785,0.0606,0.0770,1.5540
gbr,Gradient Boosting Regressor,0.0966,0.0314,0.1769,0.9821,0.0580,0.1509,13.2600
knn,K Neighbors Regressor,0.1185,0.0687,0.2619,0.9608,0.1059,0.3337,1.7960
ada,AdaBoost Regressor,0.3493,0.1776,0.4210,0.8986,0.1575,0.7372,13.8540
lr,Linear Regression,0.8369,1.2594,1.1099,0.2754,0.3724,1.4224,1.8780


In [9]:
from sklearn.metrics import mean_absolute_error
trained_models = []
weighted_mae_results = []
for model in best:
    finalize = s.finalize_model(model)
    trained_models.append(finalize)



In [10]:
weighted_mae_results=[]
for trained_model in trained_models:
    y_pred_test = trained_model.predict(X_test)
    y_pred_train = trained_model.predict(X_train)
    weighted_mae_test = mean_absolute_error(y_test,y_pred_test,sample_weight=sample_weight['weight'].loc[y_test.index])
    weighted_mae_train = mean_absolute_error(y_train,y_pred_train,sample_weight=sample_weight['weight'].loc[y_train.index])
    test_train_ratio = weighted_mae_test/weighted_mae_train
    weighted_mae_results.append({
        'Model': type(trained_model.steps[-1][-1]).__name__,
        'test-Weighted_MAE': weighted_mae_test,
        'train-Weighted_MAE': weighted_mae_train,
        'test_train_ratio': test_train_ratio
    })
    

In [11]:
results_df = pd.DataFrame(weighted_mae_results).sort_values(by='test-Weighted_MAE')
results_df.round(6)

,Model,test-Weighted_MAE,train-Weighted_MAE,test_train_ratio
0,ExtraTreesRegressor,0.120187,0.000822,146.242731
1,RandomForestRegressor,0.132186,0.051331,2.575159
2,CatBoostRegressor,0.135922,0.075812,1.792894
3,LGBMRegressor,0.144314,0.089592,1.610800
4,DecisionTreeRegressor,0.172183,0.000822,209.511804
5,GradientBoostingRegressor,0.188651,0.165043,1.143045
6,KNeighborsRegressor,0.228877,0.175834,1.301661
7,AdaBoostRegressor,0.351942,0.341124,1.031712
8,LinearRegression,0.578754,0.551379,1.049648
